In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from itertools import count
from IPython.display import clear_output
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

## CartPole-v0 환경을 이용, 폴이 안쓰러지게 학습
env = gym.make('CartPole-v0').unwrapped

plt.ion()

In [2]:
## Training 중인 상태를 plot으로 표현

episode_durations = []


def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    plt.axhline(y=200,linestyle ='--')
    # 100개의 에피소드 평균을 가져 와서 도표 그리기
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # 도표가 업데이트되도록 잠시 멈춤
    

In [3]:
## Experience Replay
## 학습에 이용할 메모리
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, transition):
        """transition 저장"""
        if len(self.memory) < self.capacity:
            self.memory.append(transition)
        self.position = (self.position + 1) % self.capacity
        if len(self.memory) >= self.capacity: 
            self.memory[self.position] = transition

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
## 큰 차원의 데이터를 다루기위해서 사용


## 학습에 쓰이는  NN 
## Input은 State가 되고 Output은 Q - value가 된다.
## 마지막에서 4개의 Q - Value를 Output으로 가진다. 
class DQN(nn.Module):

    def __init__(self, input_size, hidden_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size*2)
        self.fc3 = nn.Linear(hidden_size*2,action_size)
        self.fc4 = nn.Linear(hidden_size*2,action_size)
        self.fc5 = nn.Linear(hidden_size*2,action_size)
        self.fc6 = nn.Linear(hidden_size*2,action_size)
        
    ## 최적화 중에 다음 행동을 결정하기 위해서 하나의 요소 또는 배치를 이용해 호촐됨.
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x1 = self.fc3(x)
        x2 = self.fc4(x)
        x3 = self.fc5(x)
        x4 = self.fc6(x)
        return x1,x2,x3,x4
      

In [5]:
## Hyperparameters
'''OFFLINE 학습 방법에서는 Exploration을 최대한 다양하게 해야된다'''
BATCH_SIZE = 30
GAMMA = 0.99
EPS_START = 0.99
EPS_END = 0.5
EPS_DECAY = 400
TARGET_UPDATE = 10
num_episodes = 1000
hidden_size = 50
tau = 0.1

train_num = 20000

## 메모리 크기 / Offline Learning이기 때문에 메모리의 크기가 크다.
memory = ReplayMemory(1000000)

## gym 행동 공간에서 행동의 숫자를 얻기.
input_size = env.observation_space.shape[0]
n_actions = env.action_space.n

## 큰 차원의 데이터를 처리하기 위해서 NN을 사용
policy_net = DQN(input_size,hidden_size, n_actions)
target_net = DQN(input_size,hidden_size, n_actions)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

## 최적화 기준 RMS(Root Mean Squeare)
optimizer = optim.RMSprop(policy_net.parameters(), lr =0.001)

## 몇번의 스탭을 했는지.
steps_done = 0

In [6]:
'''Action을 하는 Q 값을 랜덤으로 설정하기 위한변수'''
random_q = [1,2,3,4]


## e- greedy를 이용한 Action 선택
def select_action(state):
    global steps_done
    ## 0과1사이의 값을 무작위로 가져옴
    sample = random.random()
    ## Epsilon-Threshold의 값이 step이 반복될 수록 작아진다.
    ## 점점 greedy action을 택하게 만듬.
    steps_done += 1
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
    math.exp(-1. * (steps_done) / EPS_DECAY)
    if sample > eps_threshold :
        with torch.no_grad():
            """
            t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            기대 보상이 더 큰 행동을 선택할 수 있습니다.
            """
            '''4개의 Q 값 중 랜덤하게 하나를 선택하는 구조'''
            act1,act2,act3,act4 = policy_net(Variable(state))
            sample_num = random.choice(random_q)
            if sample_num == 1 :
                action = torch.FloatTensor(act1).data.max(1)[1].view(1,1)
            elif sample_num == 2:
                action = torch.FloatTensor(act2).data.max(1)[1].view(1,1)
            elif sample_num == 3 :
                action = torch.FloatTensor(act3).data.max(1)[1].view(1,1)
            else :
                action = torch.FloatTensor(act4).data.max(1)[1].view(1,1)
            '''
            ## Deterministic하게 선택하는 구조
            act,_,_,_ = policy_net(Variable(state))
            action = torch.FloatTensor(act).data.max(1)[1].view(1,1)
            '''     
            
            
            return action
        # Random action 선택
    else:
        return torch.LongTensor([[random.randrange(n_actions)]])
    
    
    '''Test 할때 사용되는 방법으로 Majority Action을 찾는다. 4개의 Q Value가 투표형식으로 Action을 선택한다.'''
def select_majority_action(act1,act2,act3,act4):
    ## Majority Vote
    action1 = torch.FloatTensor(act1).data.max(1)[1].view(1,1)
    action2 = torch.FloatTensor(act2).data.max(1)[1].view(1,1)
    action3 = torch.FloatTensor(act3).data.max(1)[1].view(1,1)
    action4 = torch.FloatTensor(act4).data.max(1)[1].view(1,1)
    ## Cartpole의 Action Dim = 2 (0,1) 이므로 2가지만 생각함.
    count_zero = 0
    if action1 == 0 :
        count_zero += 1
    if action2 == 0 :
        count_zero += 1
    if action3 == 0 :
        count_zero += 1
    if action4 == 0 :
        count_zero += 1

    if count_zero >= 2 :
        action = 0
    else :
        action = 1
    return action


In [7]:
'''DATASET을 모으면서 학습하기 위한 DQN OPTIMZE MODEL  //// 굳이 사용하지 않아도 됨'''

## Optimize
def optimize_model_DQN():
    if len(memory) < BATCH_SIZE:
        return
    ## Memory에서 Sample을 가져옴    
    transitions = memory.sample(BATCH_SIZE)
    
    ## Batch로 나눔.
    state_batch, action_batch, next_state_batch, reward_batch, done_batch = zip(*transitions)
    
    ## Sample에서 나온 Data를 각각 모음.
    state_batch = torch.cat(state_batch)
    next_state_batch = Variable(torch.cat(next_state_batch))
    action_batch = Variable(torch.cat(action_batch))
    reward_batch = Variable(torch.cat(reward_batch))
    done_batch = Variable(torch.cat(done_batch))
    

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_action_values = policy_net(next_state_batch).max(1)[0]
    ## 기대 Q 값 계산
    expected_state_action_values = reward_batch + (GAMMA * next_state_action_values)*(1-done_batch)
    

    ## Huber 손실 계산 / L1 loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    ## 모델 최적화
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()


In [8]:
## Optimize
'''총 4개의 Q Value로 나온 LOSS의 평균 구하기'''
'''LOSS는 DQN 4개가 나옴'''
'''My way  '''
def optimize_model_EnsembleDQN1():
        if len(memory) < BATCH_SIZE:
            return
        ## Memory에서 Sample을 가져옴    
        transitions = memory.sample(BATCH_SIZE)

        ## Batch로 나눔.
        state_batch, action_batch, next_state_batch, reward_batch, done_batch = zip(*transitions)

        ## Sample을 각각의 데이터로 모음.
        state_batch =torch.cat(state_batch)
        next_state_batch = Variable(torch.cat(next_state_batch))
        action_batch = Variable(torch.cat(action_batch))
        reward_batch = Variable(torch.cat(reward_batch))
        done_batch = Variable(torch.cat(done_batch))


        '''Optimize 하는 부분 / /
        DQN 으로 Loss를 계산함'''

        ## 공통적으로 사용하는 Q - Value // 4개의 Q - Value가 나옴.
        state_action_values1,state_action_values2,\
        state_action_values3,state_action_values4 = policy_net(state_batch)

        state_action_values1 = state_action_values1.gather(1, action_batch)
        state_action_values2 = state_action_values2.gather(1, action_batch)
        state_action_values3 = state_action_values3.gather(1, action_batch)
        state_action_values4 = state_action_values4.gather(1, action_batch)
        
        next_state_action_values1,next_state_action_values2,next_state_action_values3,\
        next_state_action_values4 = policy_net(next_state_batch)

        
        '''DQN 계산하는 부분'''
        
        ## DQN Loss 계산

        next_state_values1 = next_state_action_values1.max(1)[0]
        next_state_values2 = next_state_action_values2.max(1)[0]
        next_state_values3 = next_state_action_values3.max(1)[0]
        next_state_values4 = next_state_action_values4.max(1)[0]

        
        
        
        '''첫번째 Q Value'''
        ## 기대 Q 값 계산 
        ## Huber 손실 계산 / L1 loss
        dqn_expected_state_action_values1 = reward_batch + (GAMMA * next_state_values1)*(1-done_batch)
        dqnloss1 = F.smooth_l1_loss(state_action_values1, dqn_expected_state_action_values1.unsqueeze(1))

        '''두번째 Q Value'''
        dqn_expected_state_action_values2 = reward_batch + (GAMMA * next_state_values2)*(1-done_batch)
        dqnloss2 = F.smooth_l1_loss(state_action_values2, dqn_expected_state_action_values2.unsqueeze(1))
        
        '''세번째 Q Value'''
        dqn_expected_state_action_values3 = reward_batch + (GAMMA * next_state_values3)*(1-done_batch)
        dqnloss3 = F.smooth_l1_loss(state_action_values3, dqn_expected_state_action_values3.unsqueeze(1))
        
        '''네번째 Q Value'''
        dqn_expected_state_action_values4 = reward_batch + (GAMMA * next_state_values4)*(1-done_batch)
        dqnloss4 = F.smooth_l1_loss(state_action_values4, dqn_expected_state_action_values4.unsqueeze(1))
        



        ## 계산해서 나온 모든 LOSS의 평균으로 업데이트함.
        loss = (dqnloss1+dqnloss2+dqnloss3+dqnloss4) / 4
        ## 모델 최적화
        optimizer.zero_grad()
        loss.backward()
        for param in policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()


In [9]:
## Optimize
'''총 4개의 Q Value로 나온 Q Value의 평균을 이용해서 LOSS 구하기'''
'''LOSS는 DQN으로 한개가 나옴'''
'''Google이 말한 방식을 코드로 표현해봄 (Modified) '''
def optimize_model_EnsembleDQN2():
        if len(memory) < BATCH_SIZE:
            return
        ## Memory에서 Sample을 가져옴    
        transitions = memory.sample(BATCH_SIZE)


        state_batch, action_batch, next_state_batch, reward_batch, done_batch = zip(*transitions)


        state_batch =torch.cat(state_batch)
        next_state_batch = Variable(torch.cat(next_state_batch))
        action_batch = Variable(torch.cat(action_batch))
        reward_batch = Variable(torch.cat(reward_batch))
        done_batch = Variable(torch.cat(done_batch))


        '''Optimize 하는 부분 / /
        DQN Loss를 계산함.'''

        ## 공통적으로 사용하는 Q - Value
        state_action_values1,state_action_values2,\
        state_action_values3,state_action_values4 = policy_net(state_batch)
        
        
        
        ## 4개의 Q - Value의 평균을 이용해서 LOSS를 계산함.
        mean_q_value = (state_action_values1+state_action_values2+state_action_values3+state_action_values4)/4

        mean_state_action_values = mean_q_value.gather(1, action_batch)
   
        
        next_state_action_values1,next_state_action_values2,next_state_action_values3,\
        next_state_action_values4 = policy_net(next_state_batch)
        
        mean_next_state_action_values = (next_state_action_values1+next_state_action_values2+\
                                         next_state_action_values3+next_state_action_values4)/4
        
        '''DQN 계산하는 부분'''
        
        ## DQN Loss 계산

        next_state_action_values = mean_next_state_action_values.max(1)[0]
    
        '''평균 Q Value'''
        ## 기대 Q 값 계산 
        ## Huber 손실 계산 / L1 loss
        dqn_expected_state_action_values = reward_batch + (GAMMA * next_state_action_values)*(1-done_batch)
        dqnloss = F.smooth_l1_loss(mean_state_action_values, dqn_expected_state_action_values.unsqueeze(1))
        


        ## DQN에서 나온 LOSS의 평균을 이용
        loss = dqnloss
        
        ## 모델 최적화
        optimizer.zero_grad()
        loss.backward()
        for param in policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()


In [ ]:
## 폴이 쓰러지면 한번에 에피소드가 종료
## Collecting Replay(Memory)

'''Offline Learning 방법'''
'''메모리 저장하는 부분'''
for t in count():
    ## 환경과 상태 초기화
    state = env.reset()
    
    ## 현재 상태를 지정
    for t in count():
        ## 행동 선택과 수행
        state = torch.FloatTensor([state])
        
        action = select_action(state)
        
        next_state, reward, done, _ = env.step(action.item())
               
        if done : 
            reward = -100
        ## 메모리에 변이 저장(한번에 에피소드에 많은 메모리가 생김)
        memory.push((state,action, torch.FloatTensor([next_state]), \
                    torch.FloatTensor([reward]), torch.FloatTensor([done])))
            
        if done or t > 500:
            '''DATA를 모으면서 학습하는 구조를 원한다면 학습 중인 과정을 볼수 있게
            하는 부분'''
            #episode_durations.appent(t+1)
            break 
        
        
        ## 다음 상태로 이동
        state = next_state
        '''OFFLINE LEARNING 방법에서는 Episode를 통해 Dataset을 쌓으면서 학습이
        가능하다.'''
        '''##Data를 모으면서 학습하는 구조 : IF U WANT
        
        optimize_model_DQN()
        
    if i_episode % 100 == 0:
        plot_durations()
    if i_episode % 1000 == 0 :
        clear_output()
        
        '''
        
    
    if len(memory) >= memory.capacity :
            break


In [ ]:
## Training Section
'''Replay에 있는 모든 데이터를 이용해서 학습한다 /
    Optimize 하는 부분'''
for t in range(train_num) : 
    '''두 개의 Optimize 방법 중 원하는 것을 사용하면 됨.'''
    optimize_model_EnsembleDQN1()
    #optimize_model_EnsembleDQN2()
    
    if t % TARGET_UPDATE == 0:
        for param, target_param in zip(policy_net.parameters(),target_net.parameters()):
                target_param.data.copy_(tau * param.data + (1-tau) * target_param.data)
                
                
                
target_net.load_state_dict(policy_net.state_dict())

In [ ]:
## Testing Section
results = []
num_episodes =100
i_episode = 0
for i_episode in range(num_episodes):
    ## 환경과 상태 초기화
    state = env.reset()
    
    ## 에피소드 시작
    for t in count():
         ## 행동 선택과 수행 / Test하는 과정이므로 학습된 네트워크가 행동 결정
        state = torch.FloatTensor([state])
        
        
        act1,act2,act3,act4 = target_net(Variable(state))
        
        action = select_majority_action(act1,act2,act3,act4)
        
        next_state, reward, done, _ = env.step(action)
               
      
        if done or t > 500:
            results.append(t + 1)
            break 
        
        
        ## 다음 상태로 이동
        state = next_state


def plot_durations1():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(results, dtype=torch.float)
    plt.title('Test')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    plt.axhline(y=200,linestyle ='--')
    # 100개의 에피소드 평균을 가져 와서 도표 그리기
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # 도표가 업데이트되도록 잠시 멈춤
    
    
    
    
plot_durations1()
print('Complete')

plt.ioff()
plt.show()

In [ ]:
'''Optimize Ensemble DQN2'''
''' Train 1 Time'''
plot_durations1()

In [ ]:
'''Optimize Ensemble DQN2'''
''' Train 2 Times'''
plot_durations1()

In [ ]:
''' Optimize Ensemble DQN1'''
''' Train 1 Times'''
plot_durations1()

In [ ]:
''' Optimize Ensemble DQN1'''
''' Train 2 Times'''
plot_durations1()

Comments : 코드는 학습을 하지만, 학습 결과를 더 좋게 만드는 방법이 많이 있는 것 같다. OFFLINE LEARNING이다 보니, OFF-POLICY 방법을 사용했고, 그중에서도 ENSEMBLE DQN을 사용했다. 여러개의 Q Value를 이용한 방법이다. 확실히 데이터셋이 크고 조금만 더 수정하면 좋은 결과가 나올 것 같다. 같은 데이터셋을 통해서 OFF-Policy 알고리즘을 비교해 볼 수 있다. Ensemble Online과 마찬가지로 2개의 Optimize 방법을 만들어 보았다.